In [50]:
from pathlib import Path
import json
from json import JSONDecodeError
from datetime import datetime, timezone
from pprint import pprint
from mt.resources import keys
import requests

The thesis is that the average pnl of the launchpad projects is the most obvious way to value the launchpad token (assuming that the launchpad token grants access to the launches) because people will see which launchpad is producing the most winners and that will drive demand for the token, so it can be used to derive a relative valuation and therefore show which projects are overvalued or undervalued on that basis.

In [31]:
fund = 1000 # update this with current fund value
withdrew = 0 # how much cash withdrawn this rebalance period (so i can correct for this when calculating pnl)
current_data = [
    {'name': 'seedify', 'pnl': 128.1, 'mcap': 70176940, 'fdv': 120_759_052},
    {'name': 'enjinstarter', 'pnl': 71.8, 'mcap': 18788146, 'fdv': 25_009_249},
    {'name': 'trustswap', 'pnl': 62.7, 'mcap': 12000323, 'fdv': 12_000_323},
    {'name': 'poolz_finance', 'pnl': 38.1, 'mcap': 15987043, 'fdv': 18_618_291},
    {'name': 'bullstarter', 'pnl': 36.3, 'mcap': 385356, 'fdv': 15_667_073}
]

In [49]:
records_path = Path("/home/ross/coding/modular_trader/records/launchpad_rebalancer")
records_path.mkdir(parents=True, exist_ok=True)
records_path = records_path / 'lp_rebal.json'
try:
    with open(records_path, 'r') as file:
        records = json.load(file)
except (FileNotFoundError, JSONDecodeError):
    records = dict()

if records:
    sorted_keys = list(sorted(records.keys()))
    most_recent = records[sorted_keys[-1]]
    prev_total = most_recent['total_value']
else:
    prev_total = fund

period_pnl = round((fund - prev_total) / prev_total, 5)

['1699357368', '1699357669', '1699357807']

In [44]:
for ido in current_data:
    ido['score'] = round(100000 * ido['pnl'] / ido['fdv'], 5)

In [45]:
scores = [ido['score'] for ido in current_data]

In [46]:
for ido in current_data:
    ido['allocation_pct'] = round(ido['score'] / sum(scores), 5)
    ido['allocation_usdt'] = round(ido['allocation_pct'] * fund, 5)
    print(f"{ido['name']} allocation: ${ido['allocation'] * fund:.2f}")

seedify allocation: $78.46
enjinstarter allocation: $212.35
trustswap allocation: $386.46
poolz_finance allocation: $151.36
bullstarter allocation: $171.37


In [47]:
timestamp = round(datetime.now(tz=timezone.utc).timestamp())
new_record = dict(
    total_value=fund,
    asset_1=current_data[0],
    asset_2=current_data[1],
    asset_3=current_data[2],
    asset_4=current_data[3],
    asset_5=current_data[4],
)

records[timestamp] = new_record

with open(records_path, 'w') as file:
    json.dump(records, file)

In [48]:
pprint(records)

{1699357807: {'asset_1': {'allocation': 0.07846124784915337,
                          'allocation_pct': 0.07846,
                          'allocation_usdt': 78.46,
                          'fdv': 120759052,
                          'mcap': 70176940,
                          'name': 'seedify',
                          'pnl': 128.1,
                          'score': 0.10608},
              'asset_2': {'allocation': 0.21234868203155893,
                          'allocation_pct': 0.21234,
                          'allocation_usdt': 212.34,
                          'fdv': 25009249,
                          'mcap': 18788146,
                          'name': 'enjinstarter',
                          'pnl': 71.8,
                          'score': 0.28709},
              'asset_3': {'allocation': 0.38645629089443223,
                          'allocation_pct': 0.38646,
                          'allocation_usdt': 386.46,
                          'fdv': 12000323,
                  

In [53]:
endpoint = "https://api.cryptorank.io/v1/currencies? api_key=keys.cr_key"
data = requests.get(endpoint)
data

<Response [401]>